## 🪄 Install `raga-testing-platform` library


In [ ]:
pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple raga-testing-platform==1.1.55

### Put your `ACCESS KEY & SECRET KEY`

---


In [ ]:
# Define the access key, secret key, and host
ACCESS_KEY = "Paste secret key"
SECRET_KEY = "Paste access key"
HOST = "https://backend.platform.raga.ai"

### Import All raga lib from raga module


In [ ]:
from raga import *
import datetime

### 🪄 Set up a Run


In [ ]:
run_name = "Defect_Detection_Tests"

### 👟 Run Data Drift Detection Test


In [ ]:
test_session = TestSession(
    project_name="Defect Detection",
    run_name=run_name,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)

train_dataset_name = "screw_train_original1"
test_dataset_name = "screw_train_original_bg"

rules = DriftDetectionRules()
threshold = 30
rules.add(type="single_class_anomaly_detection", dist_metric="Mahalanobis", _class="ALL", threshold=threshold)

#To Run OD Test
edge_case_detection = data_drift_detection(test_session=test_session,
                                           test_name=f"baker_hughes_drift_v2",
                                           train_dataset_name=train_dataset_name,
                                           field_dataset_name=test_dataset_name,
                                           train_embed_col_name="ImageVectorsM1",
                                           field_embed_col_name = "ImageVectorsM1",
                                           level = "image",
                                           rules = rules)


test_session.add(edge_case_detection)

test_session.run()

### Run Near Duplicates Test


In [ ]:
test_session = TestSession(
    project_name="Defect Detection",
    run_name=run_name,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)

rules = LQRules()
rules.add(metric="similarity_score", metric_threshold=0.99995)


train_dataset_name = "pcb1_train_dataset_v1"
edge_case_detection = nearest_duplicate(test_session=test_session,
                                          dataset_name = train_dataset_name,
                                          test_name = "Near-Duplicates-v1",
                                          type = "near_duplicates",
                                          output_type="near_duplicates",
                                          embed_col_name="ImageEmbedding",
                                          rules=rules)

test_session.add(edge_case_detection)

test_session.run()

###Run Data Leakage Test


In [ ]:
test_session = TestSession(
    project_name="Defect Detection",
    run_name=run_name,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)

train_dataset_name_ = "cable_train_dataset_v1"
dataset_name_ = "cable_val_dataset_v1"

rules = LQRules()
rules.add(metric = 'overlapping_samples', metric_threshold = 0.97)


edge_case_detection = data_leakage_test(test_session=test_session,
                                           test_name="dataleakage_baker_hughes_v2",
                                           train_dataset_name=train_dataset_name_,
                                           dataset_name=dataset_name_,
                                           type = "data_leakage",
                                           output_type="image_data",
                                           train_embed_col_name="ImageEmbedding",
                                           embed_col_name = "ImageEmbedding",
                                           rules = rules)

test_session.add(edge_case_detection)

test_session.run()

### Run Labelling Quality Test


In [ ]:
test_session = TestSession(
    project_name="Defect Detection",
    run_name=run_name,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)

dataset_name = "mistake_score_cable_2"

rules = LQRules()
rules.add(metric="mistake_score", label=["ALL"], metric_threshold=0.85)


edge_case_detection = labelling_quality_test(test_session=test_session,
                                            dataset_name = dataset_name,
                                            test_name = "Labeling Quality Test2",
                                            type = "labelling_consistency",
                                            output_type="instance_segmentation",
                                            mistake_score_col_name = "MistakeScores",
                                            gt="GT",
                                            model="GT",
                                            rules = rules)
test_session.add(edge_case_detection)

test_session.run()

### Run Scenario Imbalance Test


In [ ]:
test_session = TestSession(
    project_name="Defect Detection",
    run_name=run_name,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)

augmented_dataset_name = "screw_train_all_aug"

rules = SBRules()
rules.add(metric="js_divergence", ideal_distribution="uniform", metric_threshold=0.1)
rules.add(metric="chi_squared_test", ideal_distribution="uniform", metric_threshold=0.1)

edge_case_detection = scenario_imbalance(test_session=test_session,
                                            dataset_name = augmented_dataset_name,
                                            test_name = "Scenario_Imbalance4",
                                            type = "scenario_imbalance",
                                            output_type="metadata",
                                            rules = rules,
                                            aggregationLevels=["Augmentation"]
                                             )

test_session.add(edge_case_detection)

test_session.run()

### Run Failure Mode Analysis Cluster Level


In [ ]:
test_session = TestSession(
    project_name="Defect Detection",
    run_name=run_name,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)

augmented_dataset_name = "cable_val_original"

rules = FMARules()
rules.add(metric='F1Score',  metric_threshold=0.1, label='ALL', conf_threshold=0.1, iou_threshold=0.1)

cls_default = clustering(test_session=test_session,
                         dataset_name = augmented_dataset_name,
                         method="k-means",
                         embedding_col="ImageVectorsM1",
                         level="image",
                         args= {"numOfClusters": 5},
                         force=True)


edge_case_detection = failure_mode_analysis(test_session=test_session,
                                            dataset_name = augmented_dataset_name,
                                            test_name = "FMA_Clustering_2",
                                            model = "yolov8",
                                            gt = "GT",
                                            rules = rules,
                                            output_type="instance_segmentation",
                                            type="fma",
                                            clustering=cls_default,
                                            embedding_col_name="ImageVectorsM1",
                                            )

test_session.add(edge_case_detection)

test_session.run()

In [ ]:
test_session = TestSession(
    project_name="Defect Detection",
    run_name=run_name,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)

augmented_dataset_name = "screw_train_original_bg"

rules = FMARules()
rules.add(metric='F1Score',  metric_threshold=0.4, label='ALL', conf_threshold=0.3, iou_threshold=0.3)
rules.add(metric='Recall',  metric_threshold=0.4,  label='ALL', conf_threshold=0.3, iou_threshold=0.3)
rules.add(metric='Precision',  metric_threshold=0.4, label='ALL', conf_threshold=0.3, iou_threshold=0.3)

cls_default = clustering(test_session=test_session,
                         dataset_name = augmented_dataset_name,
                         method="k-means",
                         embedding_col="ImageVectorsM1",
                         level="image",
                         args= {"numOfClusters": 8},
                         force=True)


edge_case_detection = failure_mode_analysis(test_session=test_session,
                                            dataset_name = augmented_dataset_name,
                                            test_name = "FMA_Clustering_3",
                                            model = "yolov8",
                                            gt = "GT",
                                            rules = rules,
                                            output_type="instance_segmentation",
                                            type="fma",
                                            clustering=cls_default,
                                            embedding_col_name="ImageVectorsM1",
                                            )

test_session.add(edge_case_detection)

test_session.run()

### Run Class Imbalance Test


In [ ]:
test_session = TestSession(
    project_name="Defect Detection",
    run_name=run_name,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    host=HOST
)

rules = ClassImbalanceRules()
rules.add(metric="js_divergence", ideal_distribution="uniform", metric_threshold=0.10, label="ALL")
rules.add(metric="chi_squared_test", ideal_distribution="uniform", metric_threshold=0.10, label="ALL")

dataset_name = "pcb_1_train_original"
distribution_test = class_imbalance_test(test_session=test_session,
                                         dataset_name=dataset_name,
                                         test_name="Class Imbalance Test3",
                                         type="class_imbalance",
                                         output_type="instance_segmentation",
                                         annotation_column_name="AnnotationsV1",
                                         rules=rules)

test_session.add(distribution_test)
test_session.run()